In [0]:
dbutils.fs.cp("file:/Workspace/Shared/employees.csv","dbfs:/FileStore/employees.csv")


True

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/products.json","dbfs:/FileStore/products.json")

True

In [0]:
# 1. Load the given CSV and JSON datasets into Databricks.
# Load the CSV file into dataframe
employees_df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/FileStore/employees.csv")
employees_df.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       102|       Alice|    Finance| 2023-02-15| 70000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
+----------+------------+-----------+-----------+------+



In [0]:
# Load the JSON file into dataframe
products_df=spark.read.format("json").load("dbfs:/FileStore/products.json")

In [0]:
# 2. Create a Delta Table using the following three methods
# Save DataFrame as Delta Table
employees_df.write.format("delta").mode("overwrite").save("/delta/employees")
# Load Delta Table
delta_employees = spark.read.format("delta").load("/delta/employees")
# Show Delta Table
delta_employees.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       102|       Alice|    Finance| 2023-02-15| 70000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
+----------+------------+-----------+-----------+------+



In [0]:
# Save products DataFrame as Delta Table
products_df.write.format("delta").mode("overwrite").save("/delta/products")
delta_products = spark.read.format("delta").load("/delta/products")

In [0]:
%sql
-- 2. Use SQL to Create a Delta Table
-- Create Delta Table from SQL for employees
CREATE TABLE IF NOT EXISTS delta.`/delta/employees_sql`
AS SELECT * FROM csv.`dbfs:/FileStore/employees.csv`;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create Delta Table from SQL for products
CREATE OR REPLACE TABLE delta.`/delta/products_sql`
AS SELECT * FROM json.`dbfs:/FileStore/products.json`;

In [0]:
# Convert CSV to Delta (Employees)
employees_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/employees.csv")
employees_df.write.format("delta").mode("overwrite").save("/delta/employees_converted")
print("Employees csv converted to Delta format")

Employees csv converted to Delta format


In [0]:
products_df = spark.read.format("json").load("dbfs:/FileStore/products.json")
products_df.write.format("delta").mode("overwrite").save("/delta/products_converted")
print("Products json converted to Delta format")

In [0]:
# 1. Load the Delta Table for Employees Created in Task 1
employees_delta = spark.read.format("delta").load("/delta/employees")
employees_delta.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       102|       Alice|    Finance| 2023-02-15| 70000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
+----------+------------+-----------+-----------+------+



In [0]:
# 2. Creating a DataFrame for the New Employee Data
new_data = [(102, "Alice", "Finance", "2023-02-15", 75000), 
            (106, "Olivia", "HR", "2023-06-10", 65000)] 

columns = ["EmployeeID", "EmployeeName", "Department", "JoiningDate", "Salary"]

new_employees_df = spark.createDataFrame(new_data, columns)
new_employees_df.show()


+----------+------------+----------+-----------+------+
|EmployeeID|EmployeeName|Department|JoiningDate|Salary|
+----------+------------+----------+-----------+------+
|       102|       Alice|   Finance| 2023-02-15| 75000|
|       106|      Olivia|        HR| 2023-06-10| 65000|
+----------+------------+----------+-----------+------+



In [0]:
# 3. Merge the New Employee Data into the Existing Employees Delta Table
from delta.tables import *

employees_delta_table = DeltaTable.forPath(spark, "/delta/employees")

employees_delta_table.alias("employees").merge(
    new_employees_df.alias("new_data"),
    "employees.EmployeeID = new_data.EmployeeID"  
).whenMatchedUpdate(set = {
    "employees.Salary": "new_data.Salary" 
}).whenNotMatchedInsert(values = {
    "EmployeeID": "new_data.EmployeeID",  
    "EmployeeName": "new_data.EmployeeName",
    "Department": "new_data.Department",
    "JoiningDate": "new_data.JoiningDate",
    "Salary": "new_data.Salary"
}).execute()

updated_employees_df = spark.read.format("delta").load("/delta/employees")
updated_employees_df.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
|       102|       Alice|    Finance| 2023-02-15| 75000|
|       106|      Olivia|         HR| 2023-06-10| 65000|
+----------+------------+-----------+-----------+------+



In [0]:
# Explore the internals of the employees Delta table using Delta Lake features.
employees_delta_table = DeltaTable.forPath(spark, "/delta/employees")
employees_delta_table.detail().show(truncate=False)

+------+------------------------------------+----+-----------+---------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+-------------------------------------+----------------+----------------+-----------------+---------------------------------------------------------------+
|format|id                                  |name|description|location             |createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                           |minReaderVersion|minWriterVersion|tableFeatures    |statistics                                                     |
+------+------------------------------------+----+-----------+---------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+-------------------------------------+----------------+----------------+-----------------+------------------------------------------------

In [0]:
# Check the transaction history of the Delta table
history_df = employees_delta_table.history()
history_df.show(truncate=False)

+-------+-------------------+----------------+----------------------------------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Perform Time Travel and retrieve the table before the previous merge operation.
previous_version_df = spark.read.format("delta").option("versionAsOf", 1).load("/delta/employees")
previous_version_df.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
|       102|       Alice|    Finance| 2023-02-15| 75000|
|       106|      Olivia|         HR| 2023-06-10| 65000|
+----------+------------+-----------+-----------+------+



In [0]:
# Optimize the Delta table for better performance
spark.sql("OPTIMIZE delta.`/delta/employees`")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [0]:

# Optimize the Delta table and apply Z-ordering on the Department column
spark.sql("OPTIMIZE delta.`/delta/employees` ZORDER BY (Department)");

In [0]:
# Check the transaction history of the Delta table
employees_delta_table = DeltaTable.forPath(spark, "/delta/employees")
history_df = employees_delta_table.history()
history_df.show(truncate=False)

+-------+-------------------+----------------+----------------------------------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
previous_version_df = spark.read.format("delta").option("versionAsOf", 1).load("/delta/employees")
previous_version_df.show()


+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
|       102|       Alice|    Finance| 2023-02-15| 75000|
|       106|      Olivia|         HR| 2023-06-10| 65000|
+----------+------------+-----------+-----------+------+



In [0]:
# Query the Delta table at version 1
specific_version_df = spark.read.format("delta").option("versionAsOf", 1).load("/delta/employees")
specific_version_df.show()

+----------+------------+-----------+-----------+------+
|EmployeeID|EmployeeName| Department|JoiningDate|Salary|
+----------+------------+-----------+-----------+------+
|       101|        John|         HR| 2023-01-10| 50000|
|       103|        Mark|Engineering| 2023-03-20| 85000|
|       104|        Emma|      Sales| 2023-04-01| 55000|
|       105|        Liam|  Marketing| 2023-05-12| 60000|
|       102|       Alice|    Finance| 2023-02-15| 75000|
|       106|      Olivia|         HR| 2023-06-10| 65000|
+----------+------------+-----------+-----------+------+



In [0]:
# Vacuum the employees Delta table to remove old files (older than default retention of 7 days)
spark.sql("VACUUM delta.`/delta/employees`")

DataFrame[path: string]

In [0]:
# Vacuum the employees Delta table and set the retention period to 7 days
spark.sql("VACUUM delta.`/delta/employees` RETAIN 168 HOURS"); 